In [2]:
import numpy as np
import heapq
import matplotlib.pyplot as plt
import scipy.stats as sts
import random




#Code modified from session 2.1 (M/G/1 queue), modified to include manager and more than one queue

class Event:
    '''
    Store the properties of one event in the Schedule class defined below. Each
    event has a time at which it needs to run, a function to call when running
    the event, along with the arguments and keyword arguments to pass to that
    function.
    
    Attributes
    ----------
    timestamp : float
        The time at which the event should run.
    function : callable
        The function to call when running the event.
    args : tuple
        The positional arguments to pass to the function.
    kwargs : dict
        The keyword arguments to pass to the function.

    Methods
    -------
    __lt__(self, other)
        Compare two events based on their timestamp.
    run(self, schedule)
        Run the event by calling the function with the arguments and keyword


    '''
    def __init__(self, timestamp, function, *args, **kwargs):
        self.timestamp = timestamp
        self.function = function
        self.args = args
        self.kwargs = kwargs

    def __lt__(self, other):
        '''
        your docstring
        Parameters
        ----------
        other
            <include your description here>
        
        Returns
        -------
        bool
            <include your description here>
        '''
        return self.timestamp < other.timestamp

    def run(self, schedule):
        '''
        your docstring
        Parameters
        ----------
        schedule
            <include your description here>
        '''
        self.function(schedule, *self.args, **self.kwargs)


class Schedule:
    '''
    Implement an event schedule using a priority queue. You can add events and
    run the next event.
    
    The `now` attribute contains the time at which the last event was run.
    
    Attributes
    ----------
    now : float
        The time at which the last event was run.
    priority_queue : list
        The priority queue of events.

    Methods
    -------
    add_event_at(self, timestamp, function, *args, **kwargs)
        Add an event to the schedule at a specific time.
    add_event_after(self, interval, function, *args, **kwargs)
        Add an event to the schedule after a specific interval.
    next_event_time(self)
        Return the time at which the next event will run.
    run_next_event(self)
        Run the next event in the schedule.
    __repr__(self)
        Return a string representation of the schedule.
    print_events(self)
        Print the schedule and the events in the queue.


    '''
    
    def __init__(self):
        self.now = 0  
        self.priority_queue = []  
    
    def add_event_at(self, timestamp, function, *args, **kwargs):
        '''
        your docstring
        Parameters
        ----------
        <include your list and description here>
        
        Returns
        -------
        <include your list and description here>
        '''
        heapq.heappush(
            self.priority_queue,
            Event(timestamp, function, *args, **kwargs))
    
    def add_event_after(self, interval, function, *args, **kwargs):
        '''
        your docstring
        Parameters
        ----------
        <include your list and description here>
        
        Returns
        -------
        <include your list and description here>
        '''
        self.add_event_at(self.now + interval, function, *args, **kwargs)
    
    def next_event_time(self):
        return self.priority_queue[0].timestamp

    def run_next_event(self):
        '''
        your docstring
        Parameters
        ----------
        <include your list and description here>
        
        Returns
        -------
        <include your list and description here>
        '''
        event = heapq.heappop(self.priority_queue)
        self.now = event.timestamp
        event.run(self)
        
    def __repr__(self):
        return (
            f'Schedule() at time {self.now}min ' +
            f'with {len(self.priority_queue)} events in the queue')
    
    def print_events(self):
        print(repr(self))
        for event in sorted(self.priority_queue):
            print(f'  ⏱ {event.timestamp}min: {event.function.__name__}')

In [7]:
#create a grocery store simulation using the event and shedule classes

class Costumer:
    '''
    your docstring
    '''
    def __init__(self, id, arrival_time, departure_time):
        self.id = id
        self.arrival_time = arrival_time
        self.departure_time = departure_time

class Queue_MGC:
    def __init__(self, queue_id, service_distribution, manager):
        self.service_distribution = service_distribution
        self.manager = manager
        self.queue_id = queue_id
        self.arrival_times = []
        self.departure_times = []
        self.queue_length = 0
        self.costumers = []
        self.busy = False

    def __lt__(self, other): 
        #compare queues based on queue length
        return self.queue_length < other.queue_length

    def add_costumer(self, schedule, costumer):
        self.queue_length += 1
        self.costumers.append(costumer)
        print(
            f'{schedule.now:5.2f}: Add customer to queue {self.queue_id}  '
            f'Queue length: {self.queue_length}')
        self.arrival_times.append(costumer.arrival_time)
        if self.busy == False:
            schedule.add_event_after(0, self.serve_costumer)
        
    def serve_costumer(self, schedule):
        self.busy = True
        self.queue_length -= 1
        print(
            f'{schedule.now:5.2f}: Start serving customer.{self.queue_id} '
            f'Queue length: {self.queue_length}')
        service_time = self.service_distribution.rvs()
        schedule.add_event_after(service_time, self.finish_service)
        self.departure_times.append(self.arrival_times[0] + service_time)

    def finish_service(self, schedule):
        manager_probability = random.random()

        if manager_probability < 0.05:
            self.manager.add_costumer(schedule, self.departure_times[-1], self.costumers[-1])

        else:
            self.busy = False
            if self.queue_length > 0:
                schedule.add_event_after(0, self.serve_costumer)

#the manager follows a MG1 queue
class Manager:
    def __init__(self, manager_distribution):
        self.manager_distribution = manager_distribution
        self.arrival_times = []
        self.departure_times = []
        self.costumers = []
        self.queue_length = 0
        self.busy = False

    def add_costumer(self, schedule, arrival_time, costumer):
        self.queue_length += 1
        self.costumers.append(costumer)
        self.arrival_times.append(arrival_time)
        if self.queue_length == 1:
            self.serve_costumer(schedule)
        if self.busy == False:
            schedule.add_event_after(0, self.serve_costumer)

    def serve_costumer(self, schedule):
        self.busy = True
        self.queue_length -= 1
        service_time = self.manager_distribution.rvs()
        schedule.add_event_after(service_time, self.finish_service)
        self.departure_times.append(self.arrival_times[0] + service_time)

    def finish_service(self, schedule):
        self.busy = False
        if self.queue_length > 0:
            schedule.add_event_after(0, self.serve_costumer)

#The class holds the multiple queues, the manager and the costumers
class Grocery_Store:
    def __init__(self, arrival_distribution, service_distribution, manager_distribution, number_queues):
        self.number_queues = number_queues
        self.arrival_distribution = arrival_distribution
        self.service_distribution = service_distribution
        self.manager_distribution = manager_distribution
        self.manager = Manager(self.manager_distribution)
        self.queues = [Queue_MGC(i+1, self.service_distribution, self.manager) for i in range(self.number_queues)]
        self.costumer_number = 0

    def min_length_queue(self):
        #find the queue with the smallest length the ques are classed based on the __lt__ method
        return heapq.nsmallest(1, self.queues)[0]
        
    def add_costumer(self, schedule):
        self.costumer_number += 1
        arrival_time = schedule.now
        costumer = Costumer(self.costumer_number, arrival_time, None)
        min_length_queue = self.min_length_queue()
        min_length_queue.add_costumer(schedule, costumer)
        schedule.add_event_after(self.arrival_distribution.rvs(), self.add_costumer)

    def run(self, shedule):
        shedule.add_event_at(0, self.add_costumer)

def run_simulation(arrival_distribution, service_distribution, manager_distribution, number_queues, simulation_time):
    '''
    your docstring
    Parameters
    ----------
    <include your list and description here>
    
    Returns
    -------
    <include your list and description here>
    '''
    schedule = Schedule()
    grocery_store = Grocery_Store(arrival_distribution, service_distribution, manager_distribution, number_queues)
    grocery_store.run(schedule)
    while schedule.next_event_time() < simulation_time:
        schedule.run_next_event()
    return grocery_store
    



In [10]:
rho = 1/3

average_wait = rho/(2*3*(1-rho))

print(average_wait)

arrival_distribution = sts.expon(scale=1)
service_distribution = sts.uniform(3)
manager_distribution = sts.norm(5, 2)
number_queues = 2
simulation_time = 660
grocery_store = run_simulation(arrival_distribution=arrival_distribution, service_distribution=service_distribution, manager_distribution=manager_distribution, number_queues=number_queues, simulation_time=simulation_time)
sol = []
for queue in grocery_store.queues:
    sol.append(queue.queue_length)
print(f'🧍🏾‍♀️ There are {sol} people in the queue')

0.08333333333333333
 0.00: Add customer to queue 1  Queue length: 1
 0.00: Start serving customer.1 Queue length: 0
 0.98: Add customer to queue 1  Queue length: 1
 1.44: Add customer to queue 2  Queue length: 1
 1.44: Start serving customer.2 Queue length: 0
 1.93: Add customer to queue 2  Queue length: 1
 2.53: Add customer to queue 1  Queue length: 2
 3.98: Start serving customer.1 Queue length: 1
 4.87: Add customer to queue 1  Queue length: 2
 5.24: Start serving customer.2 Queue length: 0
 5.54: Add customer to queue 2  Queue length: 1
 7.46: Start serving customer.1 Queue length: 1
 8.79: Start serving customer.2 Queue length: 0
 9.75: Add customer to queue 2  Queue length: 1
10.65: Add customer to queue 1  Queue length: 2
10.68: Start serving customer.1 Queue length: 1
11.23: Add customer to queue 1  Queue length: 2
11.26: Add customer to queue 2  Queue length: 2
12.38: Add customer to queue 1  Queue length: 3
12.44: Start serving customer.2 Queue length: 1
13.85: Start serving

In [70]:
arrival_distribution = sts.expon(scale=1)
service_distribution = sts.norm(3, 1)
manager_distribution = sts.norm(5, 2)
number_queues = 3
simulation_time = 660
grocery_store = run_simulation(arrival_distribution=arrival_distribution, service_distribution=service_distribution, manager_distribution=manager_distribution, number_queues=number_queues, simulation_time=simulation_time)
sol = []
for queue in grocery_store.queues:
    sol.append(queue.queue_length)
print(f'🧍🏾‍♀️ There are {sol} people in the queue')

 0.00: Add customer to queue 1  Queue length: 1
 0.00: Start serving customer. Queue length: 0
 0.55: Add customer to queue 1  Queue length: 1
 1.43: Start serving customer. Queue length: 0
 1.44: Add customer to queue 1  Queue length: 1
 1.83: Add customer to queue 2  Queue length: 1
 1.83: Start serving customer. Queue length: 0
 1.99: Add customer to queue 2  Queue length: 1
 2.58: Add customer to queue 3  Queue length: 1
 2.58: Start serving customer. Queue length: 0
 4.09: Start serving customer. Queue length: 0
 4.79: Add customer to queue 2  Queue length: 1
 5.60: Add customer to queue 3  Queue length: 1
 5.60: Start serving customer. Queue length: 0
 5.62: Add customer to queue 3  Queue length: 1
 6.22: Add customer to queue 4  Queue length: 1
 6.22: Start serving customer. Queue length: 0
 8.13: Add customer to queue 4  Queue length: 1
 8.13: Start serving customer. Queue length: 0
 9.31: Start serving customer. Queue length: 0
 9.32: Add customer to queue 3  Queue length: 1
1

In [ ]:
#Theoretical values for a M/G/1 queue

#Average waiting time

#Average service rate
#mu = 3

# Average service time 
#tau = 1/mu

#Variance of service time
#sigma = 1

#Average arrival rate
#l = 1

#utilization
#rho = l*tau

#Average waiting time at equilibrium
#W = (rho*tau/(2*(1-rho))) * (1+ (sigma**2)/(tau**2))

#arrival_distribution = sts.expon(scale=l)
#service_distribution = sts.norm(loc=mu, scale=sigma)
#manager_distribution = sts.norm(loc=5, scale=2**0.5)
cashiers = 3
trial = 1


def comperison_wait_time():
    
    val_mu = np.linspace(0.1, 1, 10)
    #waiting_times_mean = []
    theoretical = []
    empirical = []

    for l in tqdm(val_mu):
        waiting_times = []
        for trial in range(10):
            grocery_store, _ = run_simulation(arrival_distribution= sts.expon(scale=l), service_distribution= sts.norm(loc=3, scale=1), manager_distribution=manager_distribution, queue_count=1, log_events=False)
            for queue in grocery_store.queues:
                waiting_times = waiting_times + queue.waiting_times
            #waiting_times = waiting_times + grocery_store.manager.waiting_times
        empirical.append(np.mean(waiting_times))

        mu = 3
        tau = 1/mu
        sigma = 1
        #l = 1
        rho = l*tau

        W = (rho*tau/(2*(1-rho))) * (1+ (sigma**2)/(tau**2))
        theoretical.append(W)

    plt.plot(val_mu, theoretical, color = 'red', label = r'Theoretical')
    plt.plot(val_mu, empirical, color = 'black', label = r'Simulation results')
    plt.xlabel(r'Arrival rate $\lambda$')
    plt.ylabel(r'Average waiting time (minutes)')
    plt.legend()
    plt.show()


comperison_wait_time()